In [ ]:
# 데이터 로드
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# 정규화 패키지
from sklearn.preprocessing import MinMaxScaler

In [ ]:
### 함수 정의 구간
# 데이터 업로드 _utf
def upload_data_utf(file_name):
  rtn = pd.read_csv(file_name, encoding = 'utf-8')
  rtn = pd.DataFrame(rtn)

  return rtn

def upload_data_cp(file_name):
  rtn = pd.read_csv(file_name, encoding = 'cp949', delimiter = '\t')
  rtn = pd.DataFrame(rtn)

  return rtn

### 데이터 정규화
  # Min-Max Scaling
  # 0에서 1사이로 정규화
def scaling(data, loctaion):

  # 스케일 대상 column
  scaleColumn = ['건축년도', '전용면적'] + location
  rtn = data

  # 데이터 정규화
  scaler = MinMaxScaler()
  rtn[scaleColumn] = scaler.fit_transform(rtn[scaleColumn])

  return rtn

In [ ]:
# XDATA 파일 업로드
x_data1 = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/SuwonData/XDATA(Special).csv")
x_data2 = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/XDATA(Special).csv")

x_data = pd.concat([x_data1, x_data2], ignore_index=True)

y_data1 = x_data1[['거래금액', '거래일자']]
y_data2 = x_data2[['거래금액', '거래일자']]

In [ ]:
# 계절지수 데이터 업로드
jisoo = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/계절지수.xlsx")

In [ ]:
# 날짜 타입으로 바꾸기
y_data1['거래일자'] = pd.to_datetime(y_data1['거래일자'])
y_data2['거래일자'] = pd.to_datetime(y_data2['거래일자'])
jisoo['일자'] = pd.to_datetime(jisoo['일자'])

<ipython-input-9-34b388c00f4a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data1['거래일자'] = pd.to_datetime(y_data1['거래일자'])
<ipython-input-9-34b388c00f4a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data2['거래일자'] = pd.to_datetime(y_data2['거래일자'])


In [ ]:
# 년월만 있는 '년월' 열 추가
jisoo['년월'] = jisoo['일자'].dt.to_period('M')
y_data1['년월'] = y_data1['거래일자'].dt.to_period('M')
y_data2['년월'] = y_data2['거래일자'].dt.to_period('M')

<ipython-input-10-37c232d46824>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data1['년월'] = y_data1['거래일자'].dt.to_period('M')
<ipython-input-10-37c232d46824>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data2['년월'] = y_data2['거래일자'].dt.to_period('M')


In [ ]:
# 같은 년월의 가격 지수로 거래금액 나누기
for i in range(len(y_data1)):
  for j in range(len(jisoo)):
    if y_data1['년월'][i] == jisoo['년월'][j]:
       y_data1['거래금액'][i] = float(y_data1['거래금액'][i].replace(',', '')) / jisoo['경부2권'][j] * 100

# 같은 년월의 가격 지수로 거래금액 나누기
for i in range(len(y_data2)):
  for j in range(len(jisoo)):
    if y_data2['년월'][i] == jisoo['년월'][j]:
       y_data2['거래금액'][i] = float(y_data2['거래금액'][i].replace(',', '')) / jisoo['경부1권'][j] * 100

<ipython-input-11-d48053712487>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data1['거래금액'][i] = float(y_data1['거래금액'][i].replace(',', '')) / jisoo['경부2권'][j] * 100
<ipython-input-11-d48053712487>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_data2['거래금액'][i] = float(y_data2['거래금액'][i].replace(',', '')) / jisoo['경부1권'][j] * 100


In [ ]:
# 입지 정보 칼럼명
location = ['위도','경도',
            '약국', '의원','병원', '대병원',
            '대형상권','공원', '도서관',
            '유치원', '초등학교', '중학교', '고등학교', '학원',
            '지하철',
            '약국 수', '의원 수', '병원 수', '대병원 수',
            '대형상권 수','공원 수','도서관 수',
            '유치원 수', '초등학교 수', '중학교 수', '고등학교 수', '학원 수',
            '지하철 수']

y_data = pd.concat([y_data1, y_data2], ignore_index=True)
y_data = y_data[['거래금액']]

x_data = x_data[['건축년도', '전용면적'] + location]

In [ ]:
x_data = x_data.astype('float32')
y_data = y_data.astype('float32')

In [ ]:
# 데이터 정규화
x_data = scaling(x_data, location)
x_data = x_data[['건축년도', '전용면적'] + location]

In [ ]:
# 훈련 세트와 테스트 세트로 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
# 그라디언트 부스팅 모델 생성
gb_regressor = GradientBoostingRegressor(random_state=42)

param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 7)
}

random_search = RandomizedSearchCV(estimator=gb_regressor,
                                   param_distributions=param_dist,
                                   n_iter=10, cv=5,
                                   scoring='neg_mean_squared_error',
                                   random_state=42)

random_search.fit(X_train, y_train)

# 최적의 매개변수 출력
print("Best Parameters:", random_search.best_params_)

In [ ]:
# 그라디언트 부스팅 모델 생성
gb_regressor = GradientBoostingRegressor(random_state=42)

In [ ]:
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 7)
}

In [ ]:
random_search = RandomizedSearchCV(estimator=gb_regressor, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x78b0a01b03a0>,
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78b0a01b1f60>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x78b0a11e4b80>},
                   random_state=42, scoring='neg_mean_squared_error')

In [ ]:
# 최적의 매개변수 출력
print("Best Parameters:", random_search.best_params_)

Best Parameters: {'learning_rate': 0.038573363584388155, 'max_depth': 5, 'n_estimators': 199}


In [ ]:
# 최적의 모델 추출
best_gb_model = random_search.best_estimator_

In [ ]:
# 테스트 데이터로 예측
y_pred = best_gb_model.predict(X_test)

In [ ]:
# 성능 평가
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 46594465.41208442


In [ ]:
# R-squared 계산
r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')

R-squared: 0.9372986163438085
